The the general LP that solves the problem is formulated as such:

Let $x_{gl}$ denote the amount of feed from grain farm $g$ that is used in product given to livestock farm $l$

Let $y_{g}$ denote the total amount of feed purchased from grain farm $g$

Let $z_{l}$ denote the total amount of feed sold to livestock farm $l$

max $\sum_{l \in L} r_l z_l - \sum_{g \in G} c_g y_g - \sum_{g \in G} \sum_{l \in L} t_{gl} x_{gl}$

s.t. $y_g = \sum_{l \in L} x_{gl} \hspace{50pt} \forall g \in G$

$\hspace{13pt} z_l = \sum_{g \in G} x_{gl} \hspace{50pt} \forall l \in L$

$\hspace{13pt} \sum_{g \in G} a_g x_{gl} \geq p_l z_l \hspace{36pt} \forall l \in L$

$\hspace{13pt} \sum_{g \in G} b_g x_{gl} \geq q_l z_l \hspace{36pt} \forall l \in L$

$\hspace{13pt} x_{gl} \geq 0 \hspace{76pt} \forall g \in G, l \in L$

$\hspace{13pt} 0 \leq y_g \leq s_g \hspace{61pt} \forall g \in G$

$\hspace{13pt} 0 \leq z_l \leq d_l \hspace{62pt} \forall l \in L$

In [10]:
using JuMP, Cbc, NamedArrays

# Sets
G = [1, 2, 3, 4]
L = [1, 2, 3, 4]

# Parameters
c = Dict(zip(G, [2.5, 3, 5, 4]))
a = Dict(zip(G, [10, 8, 13, 15]))
b = Dict(zip(G, [7, 12, 15, 11]))
d = Dict(zip(L, [500, 720, 580, 600]))
p = Dict(zip(L, [12, 13, 11, 10]))
q = Dict(zip(L, [10, 11, 13, 9]))
r = Dict(zip(L, [10, 13, 14, 12]))
s = Dict(zip(G, [700, 800, 800, 900]))
t = NamedArray([0.5 1 0.7 1.2; 1.1 0.8 2 1.4; 1.6 2 0.9 0.4; 0.8 0.7 1.8 1.2], (G, L), ("grain", "livestock"))

########################################
# TODO: Create and Solve General model #
########################################

using JuMP, Cbc, NamedArrays

m = Model()

@variable(m, x[G,L] >= 0) # amount of grain farm G that goes to livestock farm L
@variable(m, y[G] >= 0)  # amount of grain bought from grain farm G
@variable(m, z[L] >= 0)  # amount of food sold to the livstock farm L

@objective(m, Max, sum(r[j] * z[j] for j in L) - sum(c[i] * y[i] for i in G) - sum(t[i,j] * x[i,j] for i in G for j in L))

@constraint(m, grain[i in G], sum(x[i,j] for j in L) == y[i])
@constraint(m, livestock[j in L], sum(x[i,j] for i in G) == z[j])
@constraint(m, alfafa[j in L], sum(a[i] * x[i,j] for i in G) >= p[j] * z[j])
@constraint(m, barley[j in L], sum(b[i] * x[i,j] for i in G) >= q[j] * z[j])
@constraint(m, max_supply[i in G], y[i] <= s[i])
@constraint(m, max_feed[j in L], z[j] <= d[j])

set_optimizer(m, Cbc.Optimizer)

optimize!(m)

solution_summary(m, verbose = true)

Presolve 16 (-8) rows, 24 (0) columns and 80 (-8) elements
0  Obj -0 Dual inf 48.999996 (4)
18  Obj 18975.04
Optimal - objective value 18975.04
After Postsolve, objective 18975.04, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 18975.04008 - 18 iterations time 0.002, Presolve 0.00


* Solver : COIN Branch-and-Cut (Cbc)

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Result count       : 1
  Has duals          : false
  Message from the solver:
  "Cbc_status          = finished - check isProvenOptimal or isProvenInfeasible to see if solution found (or check value of best solution)
Cbc_secondaryStatus = unset (status_ will also be -1)
"

* Candidate solution
  Objective value      : 18975.040078201368
  Objective bound      : 18975.040078201368
  Primal solution :
    x[1,1] : 151.51515151515156
    x[1,2] : 43.63636363636357
    x[1,3] : 98.39687194525908
    x[1,4] : 406.45161290322585
    x[2,1] : 106.06060606060602
    x[2,2] : 174.54545454545465
    x[2,3] : 124.2750081459757
    x[2,4] : 116.12903225806453
    x[3,1] : 0.0
    x[3,2] : 0.0
    x[3,3] : 357.3281199087652
    x[3,4] : 77.41935483870967
    x[4,1] : 242.42424242424244
    x[4,2] : 501.8181818181818
    x[4,3] : 0.0
    x[4,4] : 0